In [1]:
using Symbolics

In [2]:
_prod(a, b, c, arrs...) = a .* _prod(b, c, arrs...)
_prod(a, b) = a .* b
_prod(a) = a

function hamiltonian_poly(z, order, inds...)
    ham = []

    if order == 0
        Num(1)
    elseif order == length(inds)
        ham = vcat(ham, _prod([z[i] for i in inds]...))
    else
        start_ind = length(inds) == 0 ? 1 : inds[end]
        for j in start_ind:length(z)
            ham = vcat(ham, hamiltonian_poly(z, order, inds..., j))
        end
    end

    return ham
end

hamiltonian_poly (generic function with 1 method)

In [3]:
function hamiltonian(z, a, order)
    ham = []

    for i in 1:order
        ham = vcat(ham, hamiltonian_poly(z, i))
    end

    sum(collect(a .* ham))
end

hamiltonian (generic function with 1 method)

In [4]:
function hamil_trig(z, a, order, trig_wave_num)
    ham = []

    # Polynomial basis
    for i in 1:order
        ham = vcat(ham, hamiltonian_poly(z, i))
    end

    # Trignometric basis
    for k = 1:trig_wave_num
        ham = vcat(ham, vcat(sin.(k*z)), vcat(cos.(k*z)))
    end

    ham = sum(collect(a .* ham))

    return ham

end

hamil_trig (generic function with 1 method)

In [5]:
function calculate_nparams(d, polyorder, usesine, trig_wave_num)
    
    # binomial used to get the combination of polynomials till the highest order without repeat, e.g nparam = 34 for 3rd order, with z = q,p each of 2 dims
    nparam = binomial(2*d + polyorder, polyorder) - 1

    if usesine == false

        return nparam

    elseif usesine == true

        # first 2 in the product formula b/c the trig basis are sin and cos i.e. two basis functions
        # 2d: b/c the phase space is two variables p,q each with 2 dims 
        trig_basis_length = 2 * trig_wave_num * 2d

        return (nparam + trig_basis_length)

    end
end

calculate_nparams (generic function with 1 method)

In [6]:
function hamilGrad_func_builder(d, usesine, polyorder, nparam, trig_wave_num)
    
    # symbolic variables
    @variables a[1:nparam]
    @variables q[1:d]
    @variables p[1:d]
    z = vcat(q,p)

    # usesine: whether to add trig basis or not
    if usesine == true

        # gives derivative of the hamiltonian, but not the skew-symmetric true one
        Dz = Differential.(z)
        ∇H_add_trig = [expand_derivatives(dz(hamil_trig(z, a, polyorder, trig_wave_num))) for dz in Dz]

        # line below makes the vector into a hamiltonian vector field by multiplying with the skew-symmetric matrix
        ∇H_trig = vcat(∇H_add_trig[d+1:2d], -∇H_add_trig[1:d])

        # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
        ∇H_eval = eval(build_function(∇H_trig, z, a)[2])

        return ∇H_eval

    elseif usesine == false

        # gives derivative of the hamiltonian, but not the skew-symmetric true one
        Dz = Differential.(z)
        f = [expand_derivatives(dz(hamiltonian(z, a, polyorder))) for dz in Dz]

        # line below makes the vector into a hamiltonian by multiplying with the skew-symmetric matrix
        ∇H = vcat(f[d+1:2d], -f[1:d])

        # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
        ∇H_eval = eval(build_function(∇H, z, a)[2])
        
        return ∇H_eval

    end

end

hamilGrad_func_builder (generic function with 1 method)

#### TESTING TIME ####

In [8]:
# d: number of variables p,q
d = 2

# use trig basis
usesine = true

" trig_wave_num can be adjusted if higher frequency arguments expected "
trig_wave_num = 10

# highest order of polynomial library function
polyorder = 3

"binomial used to get the combination of variables till the highest 
order without repeat, e.g with usesine= false, nparam = 34 for 3rd 
order, with z = q,p each of 2 dims"

nparam = calculate_nparams(d, polyorder, usesine, trig_wave_num)

114

In [16]:
# builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
∇H_eval = hamilGrad_func_builder(d, usesine, polyorder, nparam, trig_wave_num)

#17 (generic function with 1 method)

In [17]:
# wrapper function for generalized SINDY hamiltonian gradient
function hamilGradient_general!(out, z, a::AbstractVector{T}, t) where T
    ∇H_eval(out, z, a)
    return out
end

hamilGradient_general! (generic function with 1 method)

In [9]:
# 2D system with 4 variables [q₁, q₂, p₁, p₂] where q₂ = 0 and p₂ = 0
nd = 4

# 2 dims each of p and q gives 2*d = 4 variables
out = zeros(nd)

# let (a) be a vector of zeros initially of length 34 (b/c 34 is number of poly combinations for 2 variables, with 2 dims of highest order 3)

##################### NOTE: IN ACTUAL SCRIPT WE INITIALIZE THIS TO ZERO TO ALLOW BETTER OPTIMIZATION
a = ones(nparam)

x₀ = [2, 0, 0, 0]

t = 0
    

0

In [19]:
hamilGradient_general!(out, x₀, a, 0)

4-element Vector{Float64}:
  62.0
  62.0
 -18.18547032212149
 -62.0

#### VISUALIZE (a) coefficients ####

In [14]:
order = 3
trig_wave_num = 3
d = 2
nparam = calculate_nparams(d, polyorder, usesine, trig_wave_num)
# symbolic variables
@variables a[1:nparam]
@variables q[1:d]
@variables p[1:d]
z = vcat(q,p)

ham = []

# Polynomial basis
for i in 1:order
    ham = vcat(ham, hamiltonian_poly(z, i))
end

# Trignometric basis
for k = 1:trig_wave_num
    ham = vcat(ham, vcat(sin.(k*z)), vcat(cos.(k*z)))
end

ham = (collect(a .* ham))

58-element Vector{Num}:
        a[1]*q[1]
        a[2]*q[2]
        a[3]*p[1]
        a[4]*p[2]
    (q[1]^2)*a[5]
   a[6]*q[1]*q[2]
   a[7]*p[1]*q[1]
   a[8]*p[2]*q[1]
    (q[2]^2)*a[9]
  a[10]*p[1]*q[2]
                ⋮
 cos(2p[2])*a[50]
 sin(3q[1])*a[51]
 sin(3q[2])*a[52]
 sin(3p[1])*a[53]
 sin(3p[2])*a[54]
 cos(3q[1])*a[55]
 cos(3q[2])*a[56]
 cos(3p[1])*a[57]
 cos(3p[2])*a[58]